In [ ]:
# Install Ollama and Huggingface packages
%pip install llama-index-llms-ollama llama-index-embeddings-huggingface llama-index networkx llama-index-graph-stores-neo4j

In [21]:
# Use local model
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:latest", request_timeout=120)

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load documents
documents = SimpleDirectoryReader('files').load_data()

# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Build the index with local embeddings
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Set up query engine using Ollama
query_engine = index.as_query_engine(llm=llm)

# Query
response = query_engine.query("Summarize each document in a few sentences.")
print(response)


/opt/anaconda3/envs/intelSys/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Document 1:
Dogs are considered loyal companions due to their unique traits and personalities. They provide comfort, protection, and love to their owners, making them beloved pets worldwide. Whether active or relaxed, dogs bring joy to people's lives.

Document 2:
The city of Rome was the heart of the Roman Empire in ancient times. The Romans were skilled engineers who built grand structures like the Colosseum and Pantheon. They created extensive networks of roads, aqueducts, and bridges that connected their territories and left a lasting impact on modern societies.


In [28]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.indices.knowledge_graph import KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore

# Load documents
documents = SimpleDirectoryReader('files').load_data()

# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

graph_store = SimpleGraphStore()

# Build the index with local embeddings
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    graph_store=graph_store,
    embed_model='local',
    llm=llm,
    max_triplets_per_chunk=10
)

# Set up query engine using Ollama
query_engine = kg_index.as_query_engine(llm=llm)

# Query
response = query_engine.query("Summarize each document in a few sentences.")
print(response)


Document summaries cannot be provided as there are no documents to summarize.


VERSION HECTOR

In [26]:
# Use local model

from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.2:latest", request_timeout=120)

In [27]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.indices.knowledge_graph import KnowledgeGraphIndex
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.tools import QueryEngineTool

# Load documents
documents = SimpleDirectoryReader('files').load_data()


# Set up local embeddings (you can specify a different model too)
embed_model = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

# Build Knowledge Graph Index
graph_store = SimpleGraphStore()
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    graph_store=graph_store,
    embed_model='local',
    llm=llm,
    max_triplets_per_chunk=10
)

# Build the index with local embeddings
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Create query engines
kg_engine = kg_index.as_query_engine(llm=llm)
vector_engine = vector_index.as_query_engine(llm=llm)

# Combine into a Router engine
tools = [
    QueryEngineTool.from_defaults(
        query_engine=kg_engine,
        description="Use for fact-based or entity relationship queries."
    ),
    QueryEngineTool.from_defaults(
        query_engine=vector_engine,
        description="Use for general or contextual document questions."
    )
]
router_engine = RouterQueryEngine.from_defaults(query_engine_tools=tools, llm=llm)

# Query
response = router_engine.query("Summarize each document in a few sentences.")
print(response)


Document 1:
Dogs are widely regarded as loyal companions due to their unique traits and abilities. They provide comfort, protection, and unwavering love to their owners, making them beloved pets worldwide.

Document 2:
The Roman Empire's heart was the city of Rome itself, renowned for its grand architecture, engineering prowess, and extensive territorial network. The Roman Republic fostered a skilled society that left lasting impacts on art, law, and governance in modern societies today.
